In [1]:
from math import sqrt
import regex as re
import os
from glob import glob
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_recall_fscore_support, mean_squared_error
import torch
import transformers as ppb
import warnings

warnings.filterwarnings('ignore')

## Interspeech 2020 Challenge
Deadline: May 8th submission of predictions and paper.

Main Webpage: http://www.interspeech2020.org/index.php?m=content&c=index&a=lists&catid=66

Challenge Webpage: http://www.homepages.ed.ac.uk/sluzfil/ADReSS/

# Initial Plan
Only using the transcripts:
- [x] Simple clean and join all sentences, classifiy using DistillBERT, (BERT), (RoBERTa)   ## Done

## Further Feature Engineering:
### Time dimension
- Embed time total time taken - parse time blocks, take first and last
- Embed total time taken per sentence
- Time before starting speech
- Time in between each sentence
- Average / min / max / median time of sentence
- use of special characters
- number of sentences spoken

### Linguistic Features
- Embed special character tokens in speech, pauses etc. (not sure if this needed, tokenzier / and
- classify on a sentence level??
- Also use the Interviewer INV, questions / speech / time...
- Use POS Tags: as OHE vector

## Demographics
- Gender
- Age

## Fine-tuning BERT(-esque) models on spontaneous speech datasets
- fine-tune and re-classify using other spontaneous speech datasets: 

### Further work on
- Analysis of what roBERTa has actually learned in the attention heads

In [6]:
prob_ad_dir = '../train/transcription/cd/*'
controls_dir = '../train/transcription/cc/*'

In [7]:
def extract_data(file_name):
    par = {}
    par['id'] = file_name.split('/')[-1].split('.cha')[0]
    f = iter(open(file_name))
    l = next(f)
    speech = []
    try:
        curr_speech = ''
        while (True):
            if l.startswith('@ID'):
                participant = [i.strip() for i in l.split('|')]
                if participant[2] == 'PAR':
                    par['mmse'] = '' if len(participant[8]) == 0 else float(participant[8])
                    par['sex'] = participant[4][0]
                    par['age'] = int(participant[3].replace(';', ''))
            if l.startswith('*PAR:') or l.startswith('*INV'):
                curr_speech = l
            elif len(curr_speech) != 0 and not(l.startswith('%') or l.startswith('*')):
                curr_speech += l
            elif len(curr_speech) > 0:
                speech.append(curr_speech)
                curr_speech = ''
            l = next(f)
    except StopIteration:
        pass

    clean_par_speech = []
    clean_all_speech = []
    speech_time_segments = []
    is_par = False
    for s in speech:
        def _clean(s):
            speech_time_segments.append([*map(int, re.search('\x15(\d*_\d*)\x15', s).groups()[0].split('_'))])
            s = re.sub('\x15\d*_\d*\x15', '', s) # remove time block 
            s = re.sub('\[.*\]', '', s) # remove other speech artifacts [.*]
            s = s.strip()
            s = re.sub('\t|\n|<|>', '', s) # remove tab, new lines, inferred speech??, ampersand, &
            return s
        
        if s.startswith('*PAR:'):
            is_par = True
        elif s.startswith('*INV:'):
            is_par = False
            s = re.sub('\*INV:\t', '', s) # remove prefix
        if is_par:
            s = re.sub('\*PAR:\t', '', s) # remove prefix    
            clean_par_speech.append(_clean(s))
        clean_all_speech.append(_clean(s))
    
    par['speech'] = speech
    par['clean_speech'] = clean_all_speech
    par['clean_par_speech'] = clean_par_speech
    par['joined_all_speech'] = ' '.join(clean_all_speech)
    par['joined_all_par_speech'] = ' '.join(clean_par_speech)
    
    # sentence times
    par['per_sent_times'] = speech_time_segments
    par['total_time'] =  speech_time_segments[-1][1] - speech_time_segments[0][0]
    par['time_before_par_speech'] = speech_time_segments[0][0]
    par['time_between_sents'] = [0 if i == 0 else max(0, speech_time_segments[i][0] - speech_time_segments[i-1][1]) 
                                 for i in range(len(speech_time_segments))]
    return par

In [9]:
def parse_train_data():
    return _parse_data('../data/train')
    
def parse_test_data():
    return _parse_data('../data/test')

def parse_pre_train_data():
    return _parse_data('/data/train')

def _parse_data(data_dir):
    prob_ad_dir = f'{data_dir}/transcription/cd/*'
    controls_dir = f'{data_dir}/transcription/cc/*'
    
    prob_ad = [extract_data(fn) for fn in glob(prob_ad_dir)]
    controls = [extract_data(fn) for fn in glob(controls_dir)]
    controls_df = pd.DataFrame(controls)
    prob_ad_df = pd.DataFrame(prob_ad)
    controls_df['ad'] = 0
    prob_ad_df['ad'] = 1
    df = pd.concat([controls_df, prob_ad_df]).sample(frac=1).reset_index(drop=True)
    return df

In [10]:
train_df = parse_data()

In [30]:
train_df.head()

,id,mmse,sex,age,speech,clean_speech,all_clean_speech,per_sent_times,total_time,time_before_par_speech,time_between_sents,ad
0,S081,19,m,69,[*PAR:\t<well the kid> [//] the girl's laughin...,[well the kid the girl's laughin(g) at her br...,well the kid the girl's laughin(g) at her bro...,"[[0, 6166], [6166, 8700], [8700, 11360], [1136...",54117,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
1,S084,21,m,68,"[*PAR:\talright . [+ exc] 0_522\n, *PAR:\tI ...","[alright ., I see the little boy stealing cook...",alright . I see the little boy stealing cookie...,"[[0, 522], [3194, 7004], [8065, 12532], [13354...",134740,0,"[0, 2672, 1061, 822, 0, 0, 0, 0, 0, 0, 886, 0,...",0
2,S104,27,m,64,"[*PAR:\tokay, I see a boy in the cookie jar . ...","[okay, I see a boy in the cookie jar ., I see ...","okay, I see a boy in the cookie jar . I see he...","[[3209, 8329], [8329, 10258], [10258, 19851], ...",92350,3209,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
3,S101,24,f,56,[*PAR:\t(...) &um &t takin(g) some cookies . [...,"[(...) &um &t takin(g) some cookies ., and &f ...",(...) &um &t takin(g) some cookies . and &f fa...,"[[8671, 44188], [44188, 48558], [48558, 59393]...",214850,8671,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 13198, 1341, 1546]",0
4,S033,30,f,61,"[*PAR:\tmhm . [+ exc] 4672_5203\n, *PAR:\twe...","[mhm ., well the water's running over on the f...",mhm . well the water's running over on the flo...,"[[4672, 5203], [9212, 13604], [13604, 17399], ...",42523,4672,"[0, 4009, 0, 0, 0, 0, 0, 1031, 11525]",1


In [ ]:
segmented_speech = train_df.loc[:, ['clean_par_speech', 'ad', 'mmse']].apply(lambda r: pd.DataFrame({'speech_sent': r.clean_par_speech, 'ad': r.ad, 'mmse': r.mmse}), axis=1).tolist()

In [ ]:
# explode out each segment into AD / control segments
pd.concat(segmented_speech)

### Tran Base-line TfIdf / Random Forest Models / SVM Models

In [ ]:
train_df.columns

In [ ]:
random_state = 42

In [ ]:
base_lines_train_df = train_df.loc[:, ['joined_all_par_speech', 'ad', 'mmse']]

In [ ]:
# sklearn pipeline
param_space = {
    'vec__max_features': [100, 500, 1000, 10000],
#     'vec__stop_words': ['english', None],
#     'vec__analyzer': ['word', 'char'],
#     'vec__sublinear_tf': [True, False],
    'clf__n_estimators': [100, 200, 500],   # gbdt params
#     'clf__max_depth': [3, 5, 10, 20, 50],   # gbdt params
#     'clf__C': [0.1, 0.5, 1.],               # SVC params
#     'clf__kernel': ['rbf', 'sigmoid']       # SVC params
}

pipe = Pipeline([
    ('vec', TfidfVectorizer()),
    ('clf', GradientBoostingClassifier())
#     ('clf', SVC())
])
train_features, test_features, train_labels, test_labels = train_test_split(train_df.joined_all_par_speech, train_df.ad, random_state=random_state, test_size=0.2)

search = GridSearchCV(pipe, param_space, cv=5, n_jobs=6)
search.fit(train_features, train_labels)

In [ ]:
pipe.set_params(**search.best_params_)
pipe.fit(train_features, train_labels)
preds = lr_clf.predict(test_features)
print('prec, rec, f1 test', precision_recall_fscore_support(test_labels, preds))
# print('accu', )

## BERT (type) model Experimentaton

In [12]:
# For DistilBERT:
# model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

# For DistilroBERTa:
# model_class, tokenizer_class, pretrained_weights = (ppb.RobertaModel, ppb.RobertaTokenizer, 'distilroberta-base')

# BERT Base
# model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# BERT Large
# model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-large-uncased')

# roBERTa base
# model_class, tokenizer_class, pretrained_weights = (ppb.RobertaModel, ppb.RobertaTokenizer, 'roberta-base')

# roBERTa large
model_class, tokenizer_class, pretrained_weights = (ppb.RobertaModel, ppb.RobertaTokenizer, 'roberta-large')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [17]:
# Only Participant Speech
tokenized = train_df.joined_all_par_speech.apply((lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512)))
# All (INV + PAR) speech
# tokenized = train_df.joined_all_speech.apply((lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512)))

# pad so can be treated as one batch
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

# attention mask - zero out attention scores where there is no input to be processed (i.e. is padding)
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# check if multiple GPUs are available
multi_gpu = torch.cuda.device_count() > 1

if torch.cuda.is_available():
    model = model.to(device)
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
last_hidden_states = last_hidden_states[0]
if device.type == 'cuda':
    last_hidden_states = last_hidden_states.cpu()
features = last_hidden_states[:,0,:].numpy()

In [ ]:
def fit_transform(features, train_df):
    # AD classification task
    train_features, test_features, train_labels, test_labels = train_test_split(features, train_df.ad)
    parameters = {'C': np.linspace(0.0001, 100, 20)}
    grid_search = GridSearchCV(LogisticRegression(), parameters)
    grid_search.fit(train_features, train_labels)
    print('best parameters:', grid_search.best_params_)
    print('best scores: ', grid_search.best_score_)
    lr_clf = LogisticRegression(**grid_search.best_params_)
    lr_clf.fit(train_features, train_labels)
    preds = lr_clf.predict(test_features)
    print('prec, rec, f1 test', precision_recall_fscore_support(test_labels, preds))
    def cv10_avg(score):
        return round(cross_val_score(lr_clf, features, train_df.ad, cv=10, scoring=score).sum() / 10, 2)
    print(f'accu:{cv10_avg("accuracy")} prec:{cv10_avg("precision")}, rec:{cv10_avg("recall")}, f1:{cv10_avg("f1")}')
          
    # MMSE regression task
    # remove missing row
    reg_features, reg_scores = np.vstack([features[0:36], features[37:]]), train_df[train_df.mmse != ''].mmse
    train_features, test_features, train_scores, test_scores = train_test_split(reg_features, reg_scores)
    parameters = {'alpha': np.linspace(0.001, 100, 20)}
    grid_search = GridSearchCV(Ridge(), parameters)
    grid_search.fit(train_features, train_scores)
    print('best parameters:', grid_search.best_params_)
    print('best scores: ', grid_search.best_score_)
    reg_model = Ridge(**grid_search.best_params_)
    reg_model.fit(train_features, train_scores)
    preds = reg_model.predict(test_features)
    print('rmse test:', sqrt(mean_squared_error(test_scores, preds)))
    print('rmse cv:', cross_val_score(reg_model, reg_features, reg_scores, cv=10, scoring='neg_root_mean_squared_error').sum() / 10)

In [20]:
# NLP Features
bert_features = features

In [693]:
# time features
# - Embed time total time taken 
# - parse time blocks, take first and last
# - Embed total time taken per sentence
# - Time before starting speech
# - Time in between each sentence
# - Average / min / max / median time of sentence
time_dims = train_df.loc[:, ['total_time', 'time_before_par_speech', 'time_between_sents']]

In [694]:
# time_dims['avg_betweeen_sents'] = time_dims.time_between_sents.apply(lambda t: round(sum(t) / len(t)))
# time_dims['max'] = time_dims.time_between_sents.apply(max)
# time_dims['min'] = time_dims.time_between_sents.apply(min)
time_dims_eng = time_dims.drop('time_between_sents', axis=1)

In [702]:
time_features = StandardScaler().fit_transform(time_dims_eng.to_numpy())

In [ ]:
class BertForADClassification(torch.nn.Module):
    def __init__(self, bertModel, time_dims,):
        super().__init__()
        
        self.bertModel = bertModel
        self.dropout = torch.nn.Dropout(0.5)
        
        # separate Linear for OHE of time
        bert_hidden_dim = self.bertModel
    
        self.classifier = nn.Linear()
    
    def forward(input_ids, attention_mask):
        last_hidden_states = self.bertModel(input_ids, attention_mask=attention_mask)
        features = last_hidden_states[:,0,:]

In [704]:
# Concat all features
features = np.hstack([bert_features, time_features])

In [374]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.542 (+/- 0.23)
